In [175]:
PORT = 10067
NUM_PAQUETES =  1
NUM_TIPOS = 1
CONT = 0
N=15


In [176]:
"""
Esta clase define a los usuarios que esatbleceran comunicacion con el servidor. 
Cada usuario contiene un ID que servira para darle seguimiento y saber en que 
se ejecutan sus paquetes

"""

class Usuario():
    
    tipo =  "A"
    ID  = "NA"
    numPauqtes = 1
    timeSatat = 0
    message = "SIN MENSAJE"
    
    def __init__( self, tipo, numPaquetes):
        global CONT
        self.tipo = tipo
        self.paquetes =  list()
        self.timeSatat = 0
        self.ID =  CONT
        self.message = "UserID" + str(self.ID)
        for i in range(numPaquetes):
            self.paquetes.append( Paquete( self.tipo ) )
            
        CONT += 1 #Se modifica la variable global para llevar la cuenta de numero de usuarios
        
        self.imprimirUsuario()
        
    def imprimirUsuario(self):
        print( "\n Usuario ID: {} \n\t Tipo: {} \n\t Paquetes: ".format( self.ID, self.tipo))
        for i in self.paquetes:
            i.imprimirPaquete()
            
            
            
class Paquete():
    
    tiempoEjecucion = 0 #/tiempo que el servidor quedara suspendido en alguno de sus threads
    tipo = "alpha"
    ownerType =  "A"
    
    def __init__(self, owner):
        if owner  ==  "A":
            self.tipo = "alpha"
            self.tiempoEjecucion = 1
        elif owner == "B":
            self.tipo = "beta"
            self.tiempoEjecucion = 2
        elif owner == "C":
            self.tipo = "gamma"
            self.tiempoEjecucion = 3
    
    def imprimirPaquete(self):
        print( "\n\t\t **Paquete tipo {}  ||   Paquete Tiempo: {}".format( self.tipo, self.tiempoEjecucion))


    
    

In [177]:
listaUsuarios = list()

for i in range(N):
    newUser = Usuario( "A", 2)
    listaUsuarios.append(newUser)
    


 Usuario ID: 0 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 1 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 2 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 3 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 4 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 5 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 6 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 7 
	 Tipo: A 

In [178]:
import socket
import sys
from joblib import Parallel, delayed
from math import sqrt

from socket import error as SocketError
import errno
import time


def RunRequest(usuario, responsesList, dataList, messageList, sockList):
    # Create a TCP/IP socket
    # Connect the socket to the port where the server is listening
    server_address = ('localhost', PORT)
    paquetes =  usuario.paquetes
    print( "\nEste Usuario tiene {} paquetes ".format(len(paquetes)) )
    #  ----------------------------------------------------------------------------------#
    #Por cada paquete se hace una conexion al servidor para procesar cosas
    for n in range(len(paquetes)):
        print("\nPORCESANDO PAQUETE {}".format(n))
        paqueteProcesando =  paquetes[n]
        tmpEjec = paqueteProcesando.tiempoEjecucion
        try:
            print("\n INICIANDO CONEXION")
            sockList[usuario.ID] = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

            sockList[usuario.ID].connect(server_address)
            print("\n CONEXION ESTABLECIDA")

            # Send data
            messageList[usuario.ID] = usuario.message
            print('\n Sending "%s"' % usuario.message )
            #sockList[usuario.ID].send( str.encode( usuario.message ) )
            sockList[usuario.ID].send( str.encode( "{} PAQUETE {} TIEMPOEJEC {} ".format(usuario.message, n, tmpEjec) ) )
            #sockList[usuario.ID].send( str.encode( " MENSAJE 2 {}".format( usuario.message) ) )
            #sockList[usuario.ID].send( str.encode( " MENSAJE 3 {}".format( usuario.message) ) )
            #sockList[usuario.ID].send( str.encode( " MENSAJE 4 {}".format( usuario.message) ) )

            """
            Aqui deberia ir algo asi como:

            procesos = ""
            for paq in usuario.pauquetes:
                procesos +=  str(paq.tiempoEjecucion)

            sockList[usuario.ID].send( str.encode( procesos ) )

            Lo que se inetenta es que el server pueda leer los paquetes que debe recibir del
            usuario y cuanto tiempo debe tardar en procesar cada uno.

            Pensandolo bien en cada iteracion (lectura del servidor) solo se deberia enviar uno
            de los paquetes del usuario para simular el pooling e los siguientes pasos

            ==>
            try:
                paq = usuario.paquetes.pop()
                sockList[usuario.ID].send( str.encode( paq.tiempoEjecucion ) )


            else: 
                print( "Todos los pquetes han sido leido ") 
            """

            # Look for the response
            amount_received = 0
            amount_expected = 10000000
            responsesList[usuario.ID] = ''
            start=time.time()
            now=time.time()
            print("\nTRANSMISION SERVER TO CLIENT:")

            while (amount_received < amount_expected) and (start+10>now):

                dataList[usuario.ID] = sockList[usuario.ID].recv(2048).decode('utf-8')

                amount_received += len(dataList[usuario.ID])

                responsesList[usuario.ID] += dataList[usuario.ID]
                now=time.time()

            print('************ Received "%s" ************ \n' % responsesList[usuario.ID])
            sockList[usuario.ID].close()
            print("/ncheck")
            
        except SocketError as e:
            if e.errno != errno.ECONNRESET:
                f=0
                print("\nError :\n\t{}".format(e))
            else:
                f=0
                print("\nError reset")
        finally:
            sockList[usuario.ID].close()
            print("\nDONE")
            
    sockList[usuario.ID].close()
    return

responsesList=[]
dataList=[]
messageList=[]
sockList=[]
for k in range(N):
    responsesList.append("")
    dataList.append("")
    messageList.append("")
    #sockList.append(socket.socket(socket.AF_INET, socket.SOCK_STREAM))
    sockList.append("")
    
Parallel(n_jobs=N, backend="threading")(delayed(RunRequest)(listaUsuarios[k], responsesList, dataList, messageList, sockList) for k in range(N))  



Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 
Este Usuario tiene 2 paquetes 

Este Usuario tiene 2 paquetes 













PORCESANDO PAQUETE 0

PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0
PORCESANDO PAQUETE 0

PORCESANDO PAQUETE 0













 INICIANDO CONEXION

 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIANDO CONEXION
 INICIAND

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [179]:
k

14

In [180]:
for i in responsesList:
    print("\n")
    print(i)




	SERVIDOR: INICIANDO (self.port  53308): UserID0
	------SERVIDOR: CORRIENDO (53308)
		SERVIDOR: TERMINANDO (53308): UserID0



	SERVIDOR: INICIANDO (self.port  53286): UserID1
	------SERVIDOR: CORRIENDO (53286)
		SERVIDOR: TERMINANDO (53286): UserID1



	SERVIDOR: INICIANDO (self.port  53282): UserID2
	------SERVIDOR: CORRIENDO (53282)
		SERVIDOR: TERMINANDO (53282): UserID2



	SERVIDOR: INICIANDO (self.port  53288): UserID3
	------SERVIDOR: CORRIENDO (53288)
		SERVIDOR: TERMINANDO (53288): UserID3



	SERVIDOR: INICIANDO (self.port  53302): UserID4
	------SERVIDOR: CORRIENDO (53302)
		SERVIDOR: TERMINANDO (53302): UserID4



	SERVIDOR: INICIANDO (self.port  53298): UserID5
	------SERVIDOR: CORRIENDO (53298)
		SERVIDOR: TERMINANDO (53298): UserID5



	SERVIDOR: INICIANDO (self.port  53292): UserID6
	------SERVIDOR: CORRIENDO (53292)
		SERVIDOR: TERMINANDO (53292): UserID6



	SERVIDOR: INICIANDO (self.port  53290): UserID7
	------SERVIDOR: CORRIENDO (53290)
		SERVIDOR: TERMINANDO (